In [1]:
import sys

print(sys.executable)

/home/boom/.pyenv/versions/aivet/bin/python


In [2]:
import os
import torch
import fitz  # PyMuPDF
import io
from typing import List, Dict, Any, Optional
from PIL import Image

# LangChain imports
from langchain_community.chat_models import ChatDeepInfra
from langchain_ollama import ChatOllama
from langchain_community.llms import LlamaCpp
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import RetrievalQA
from langchain.schema import Document


import os
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import torch
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader

import os
import shutil
from typing import List, Optional
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, TextLoader, UnstructuredPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import torch
import chromadb
from chromadb.config import Settings

import os
import shutil
import torch
from typing import List, Optional
import chromadb
from chromadb.config import Settings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import (
    PyPDFLoader,
    TextLoader,
    UnstructuredPDFLoader,
)
from langchain.text_splitter import RecursiveCharacterTextSplitter

import os
import shutil
import torch
from typing import List, Optional
import chromadb
from chromadb.config import Settings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import (
    PyPDFLoader,
    TextLoader,
    UnstructuredPDFLoader,
)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

# HuggingFace imports for vision model
from transformers import AutoModelForCausalLM, AutoTokenizer

/home/boom/.pyenv/versions/aivet/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
# /home/boom/.pyenv/versions/aivet/bin/python -m pip install --upgrade pip
# /home/boom/.pyenv/versions/aivet/bin/python -m pip install -r requirements.txt

In [37]:
import ollama

ollama_client = ollama.Client()
ollama_client.list()["models"]

available_models = [model["model"] for model in ollama_client.list()["models"]]
available_models

['qwen2.5vl:7b',
 'llama3.2-vision:11b',
 'gemma3:12b-it-qat',
 'qwen2.5:7b',
 'gemma3:4b',
 'qwen3:8b',
 'qwen3:0.6b',
 'gemma3:27b-it-qat',
 'deepseek-r1:7b',
 'qwen2.5:14b-instruct-q4_K_M']

In [38]:
# ollama_client.pull(f"gemma3:12b-it-qat")

In [58]:
import os
import torch
from typing import List
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import (
    PyPDFLoader,
    TextLoader,
    UnstructuredPDFLoader,
)
from langchain.text_splitter import RecursiveCharacterTextSplitter


class VetDocumentProcessor:
    def __init__(self, embedding_model: str = "sentence-transformers/all-MiniLM-L6-v2"):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        self.embedder = HuggingFaceEmbeddings(
            model_name=embedding_model, model_kwargs={"device": device}
        )

    def load_documents(self, docs_path: str):
        documents = []
        for filename in os.listdir(docs_path):
            path = os.path.join(docs_path, filename)
            try:
                if filename.endswith(".pdf"):
                    loader = PyPDFLoader(path)
                    docs = loader.load()
                    if not docs or all(not d.page_content.strip() for d in docs):
                        raise ValueError("Fallback to OCR")
                elif filename.endswith(".txt"):
                    loader = TextLoader(path)
                    docs = loader.load()
                else:
                    continue
            except Exception:
                loader = UnstructuredPDFLoader(path, mode="elements")
                docs = loader.load()
            documents.extend(docs)
        return documents

    def split_and_embed(self, documents):
        splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        splits = splitter.split_documents(documents)
        texts = [doc.page_content for doc in splits]
        metadatas = [self._sanitize_metadata(doc.metadata) for doc in splits]
        embeddings = self.embedder.embed_documents(texts)
        return texts, embeddings, metadatas

    def _sanitize_metadata(self, metadata: dict) -> dict:
        return {
            k: (v if isinstance(v, (str, int, float, bool, type(None))) else str(v))
            for k, v in metadata.items()
        }


import os
import shutil
from typing import List, Optional
import chromadb
from chromadb.config import Settings


class VectorStoreManager:
    CHROMA_SETTINGS = Settings(
        allow_reset=True, anonymized_telemetry=False, is_persistent=True
    )

    def __init__(
        self,
        persist_path: str = "./chroma_db",
        collection_name: str = "vet_docs",
        embedding_model: str = "sentence-transformers/all-MiniLM-L6-v2",
    ):
        self.persist_path = persist_path
        self.collection_name = collection_name
        self.client = None
        self.collection = None
        self._init_client()

        device = "cuda" if torch.cuda.is_available() else "cpu"
        self.embedder = HuggingFaceEmbeddings(
            model_name=embedding_model, model_kwargs={"device": device}
        )

    def _init_client(self):
        try:
            self.client = chromadb.PersistentClient(
                path=self.persist_path, settings=self.CHROMA_SETTINGS
            )
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name
            )
        except ValueError as e:
            if "already exists" in str(e).lower():
                raise RuntimeError(
                    f"❌ Chroma already exists with different settings.\n"
                    f"Try deleting the folder: {self.persist_path}"
                )
            raise

    def reset(self):
        if os.path.exists(self.persist_path):
            shutil.rmtree(self.persist_path)
        self._init_client()

    def add_documents(
        self,
        texts: List[str],
        embeddings: List[List[float]],
        metadatas: Optional[List[dict]] = None,
    ):
        ids = [f"doc-{i}" for i in range(len(texts))]
        self.collection.add(
            documents=texts,
            embeddings=embeddings,
            metadatas=metadatas or [{} for _ in texts],
            ids=ids,
        )

    def query(self, query_text: str, top_k: int = 5) -> str:
        """
        Embed the query text, run similarity search, and return a plain string with
        the top-k document texts concatenated.
        """
        query_embedding = self.embedder.embed_query(query_text)
        results = self.collection.query(
            query_embeddings=[query_embedding], n_results=top_k
        )

        docs = results.get("documents", [[]])[0]
        if not docs:
            return "No results found."

        # Join the top-k document texts with two newlines for readability
        return "\n\n".join(docs)

In [ ]:
class ModelProvider:
    """Centralized model provider management for different LLM services."""

    def _load_ollama_model(self, model_name):
        """Initialize Ollama client for multimodal models."""
        try:
            import ollama

            self.ollama_client = ollama.Client()

            # Verify model exists, pull if needed
            available_models = [
                model["model"] for model in self.ollama_client.list()["models"]
            ]
            if model_name not in available_models:
                print(f"📥 Pulling Ollama model: {model_name}")
                self.ollama_client.pull(f"{model_name}")

            print(f"✅ Ollama multimodal model {model_name} ready")
        except Exception as e:
            print(f"❌ Error initializing Ollama model: {e}")
            raise

    def initialize_model(
        self, provider: str = "ollama", model_name="qwen2.5:8b", **kwargs
    ) -> Any:
        """Initialize the appropriate model based on provider."""
        provider = provider.lower()

        print(provider)
        print(model_name)
        if provider == "ollama":
            self._load_ollama_model(model_name)
            if "qwen2.5" in model_name:
                return ChatOllama(
                    model=model_name,
                    temperature=kwargs.get("temperature", 0.6),
                    top_p=kwargs.get("top_p", 0.95),
                    num_ctx=kwargs.get("num_ctx", 32768),
                )
            elif "gemma3" in model_name:
                return ChatOllama(
                    model="gemma3:8b",
                    temperature=kwargs.get("temperature", 1.0),
                    top_k=kwargs.get("top_k", 64),
                    top_p=kwargs.get("top_p", 0.95),
                    num_ctx=kwargs.get("num_ctx", 32768),
                )
        elif provider == "anthropic":

            return ChatLiteLLM(
                model="anthropic/claude-3-5-haiku-latest",
                api_key=os.getenv("ANTHROPIC_API_KEY"),
                temperature=kwargs.get("temperature", 0.6),
                max_tokens=kwargs.get("max_tokens", 8192),
            )
        elif provider == "deepinfra":
            return ChatDeepInfra(
                model_name="Qwen/Qwen2.5-72B-Instruct",
                deepinfra_api_token=os.getenv("DEEPINFRA_API_KEY"),
                max_tokens=kwargs.get("max_tokens", 8192),
                temperature=kwargs.get("temperature", 0.6),
                top_p=kwargs.get("top_p", 0.95),
                top_k=kwargs.get("top_k", 20),
            )
        elif provider == "meta":
            return ChatDeepInfra(
                model_name="meta-llama/Llama-3.3-70B-Instruct-Turbo",
                deepinfra_api_token=os.getenv("DEEPINFRA_API_KEY"),
                max_tokens=kwargs.get("max_tokens", 8192),
                temperature=kwargs.get("temperature", 0.7),
                streaming=kwargs.get("streaming", True),
            )
        elif provider == "deepseek":
            return ChatDeepInfra(
                model="deepseek-ai/DeepSeek-R1-Distill-Llama-70B",
                api_base="https://api.deepinfra.com/v1/openai",
                api_key=os.getenv("DEEPINFRA_API_KEY"),
                temperature=kwargs.get("temperature", 0.6),
                top_p=kwargs.get("top_p", 0.95),
            )
        elif provider == "local_llama":
            return LlamaCpp(
                model_name=kwargs.get("model_name", ""),
                n_ctx=kwargs.get("n_ctx", 5000),
                n_threads=kwargs.get("n_threads", 12),
                n_gpu_layers=kwargs.get("n_gpu_layers", 0),
                temperature=kwargs.get("temperature", 0.7),
                top_p=kwargs.get("top_p", 0.95),
                verbose=False,
            )
        else:
            raise ValueError(f"❌ Unsupported provider: {provider}")


class ImageToTextExtractor:
    """Modernized image-to-text extractor supporting both HuggingFace and Ollama multimodal models."""

    def __init__(
        self,
        vision_model_name: str = None,
        vision_model_provider: str = "huggingface",
        device: str = "auto",
        **kwargs,
    ):
        self.device = (
            device
            if device != "auto"
            else ("cuda" if torch.cuda.is_available() else "cpu")
        )
        self.model_name = vision_model_name
        self.model_provider = vision_model_provider.lower()
        self.model = None
        self.tokenizer = None
        self.ollama_client = None

        # Additional kwargs for model configuration
        self.model_config = kwargs

        self._load_model()

        self.extraction_query = """Task: Extract and list all blood test details from this veterinary blood test report.

        Objective:
        1. Identify and extract **chemical/analyte names** from the blood test report.
        2. Capture the **detected levels** of each chemical with their units.
        3. Extract the **normal/reference range** for each corresponding chemical.
        4. Note any flags or indicators (High, Low, Normal, etc.).

        Expected Output Format:
        - Chemical Name: [Name]
        - Detected Level: [Value with units]
        - Normal Range: [Min Value] - [Max Value with units]
        - Status: [Normal/High/Low if indicated]

        Please ensure accurate extraction, including any unit symbols (e.g., mg/dL, IU/L, g/L), and handle variations in formatting or alignment. Return results in a structured list format for easy readability."""

    def _load_model(self):
        """Load the appropriate vision model based on provider."""
        if self.model_provider == "huggingface":
            self._load_huggingface_model()
        elif self.model_provider == "ollama":
            self._load_ollama_model()
        else:
            raise ValueError(f"Unsupported model provider: {self.model_provider}")

    def _load_huggingface_model(self):
        """Load HuggingFace vision model."""
        try:
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                torch_dtype=torch.bfloat16,
                low_cpu_mem_usage=True,
                trust_remote_code=True,
                device_map="auto",
                use_cache=True,
                **self.model_config,
            )
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.model_name, trust_remote_code=True
            )
            print(f"✅ HuggingFace vision model loaded successfully on {self.device}")
        except Exception as e:
            print(f"❌ Error loading HuggingFace vision model: {e}")
            raise

    def _load_ollama_model(self):
        """Initialize Ollama client for multimodal models."""
        try:
            import ollama

            self.ollama_client = ollama.Client()

            # Verify model exists, pull if needed
            available_models = [
                model["model"] for model in self.ollama_client.list()["models"]
            ]
            if self.model_name not in available_models:
                print(f"📥 Pulling Ollama model: {self.model_name}")
                self.ollama_client.pull(f"{self.model_name}")

            print(f"✅ Ollama multimodal model {self.model_name} ready")
        except ImportError:
            raise ImportError(
                "Ollama package not found. Install with: pip install ollama"
            )
        except Exception as e:
            print(f"❌ Error initializing Ollama model: {e}")
            raise

    def clear_memory(self):
        """Clear GPU memory based on provider."""
        if self.model_provider == "huggingface":
            if self.model is not None:
                del self.model
                del self.tokenizer
                self.model = None
                self.tokenizer = None
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
        elif self.model_provider == "ollama":
            # Ollama manages its own memory, but we can clear the client reference
            self.ollama_client = None
        print("🧹 Memory cleared")

    def convert_pdf_to_images(
        self, pdf_path: str, zoom: float = 2.0
    ) -> List[Image.Image]:
        """Convert PDF pages to PIL Images."""
        images = []
        mat = fitz.Matrix(zoom, zoom)

        try:
            doc = fitz.open(pdf_path)
            for page_num in range(len(doc)):
                page = doc.load_page(page_num)
                pix = page.get_pixmap(matrix=mat)
                img_data = pix.tobytes("png")
                img = Image.open(io.BytesIO(img_data))
                images.append(img.convert("RGB"))
                print(f"📄 Converted page {page_num + 1} to image")
            doc.close()
            return images
        except Exception as e:
            print(f"❌ Error converting PDF: {e}")
            raise

    def extract_text_from_file(self, file_path: str) -> str:
        """Extract text from PDF or image file."""
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File not found: {file_path}")

        file_extension = os.path.splitext(file_path)[1].lower()
        extracted_text = ""

        try:
            if file_extension == ".pdf":
                print(f"📋 Processing PDF file: {file_path}")
                images = self.convert_pdf_to_images(file_path)
                for i, image in enumerate(images):
                    print(f"🔍 Extracting text from page {i + 1}...")
                    page_text = self._process_image(image, self.extraction_query)
                    extracted_text += f"=== Page {i + 1} ===\n{page_text}\n\n"
            elif file_extension in [".png", ".jpeg", ".jpg", ".bmp"]:
                print(f"🖼️ Processing image file: {file_path}")
                image = Image.open(file_path).convert("RGB")
                extracted_text = self._process_image(image, self.extraction_query)
            else:
                raise ValueError(f"Unsupported file type: {file_extension}")

            return extracted_text.strip()
        except Exception as e:
            print(f"❌ Error during text extraction: {e}")
            raise

    def _process_image(self, image: Image.Image, query: str) -> str:
        """Process a single image with the appropriate vision model."""
        if self.model_provider == "huggingface":
            return self._process_image_huggingface(image, query)
        elif self.model_provider == "ollama":
            return self._process_image_ollama(image, query)
        else:
            raise ValueError(f"Unsupported model provider: {self.model_provider}")

    def _process_image_huggingface(self, image: Image.Image, query: str) -> str:
        """Process image using HuggingFace model."""
        if self.model is None or self.tokenizer is None:
            raise RuntimeError(
                "HuggingFace model not loaded. Call _load_model() first."
            )

        try:
            # Prepare inputs
            inputs = self.tokenizer.apply_chat_template(
                [{"role": "user", "image": image, "content": query}],
                add_generation_prompt=True,
                tokenize=True,
                return_tensors="pt",
                return_dict=True,
            )

            inputs = inputs.to(self.device)

            gen_kwargs = {
                "max_length": 2500,
                "do_sample": True,
                "top_k": 1,
                "temperature": 0.2,
                "pad_token_id": self.tokenizer.eos_token_id,
            }

            with torch.no_grad():
                outputs = self.model.generate(**inputs, **gen_kwargs)
                outputs = outputs[:, inputs["input_ids"].shape[1] :]
                output_text = self.tokenizer.decode(
                    outputs[0], skip_special_tokens=True
                )
                return output_text.strip()
        except Exception as e:
            print(f"❌ Error processing image with HuggingFace: {e}")
            return ""

    def _process_image_ollama(self, image: Image.Image, query: str) -> str:
        """Process image using Ollama multimodal model."""
        if self.ollama_client is None:
            raise RuntimeError(
                "Ollama client not initialized. Call _load_model() first."
            )

        try:
            import base64
            from io import BytesIO

            # Convert PIL image to base64
            buffer = BytesIO()
            image.save(buffer, format="PNG")
            img_base64 = base64.b64encode(buffer.getvalue()).decode()

            # Make request to Ollama
            response = self.ollama_client.generate(
                model=self.model_name,
                prompt=query,
                images=[img_base64],
                options={
                    "temperature": self.model_config.get("temperature", 0.2),
                    "top_k": self.model_config.get("top_k", 1),
                    "top_p": self.model_config.get("top_p", 0.9),
                    "num_ctx": self.model_config.get(
                        "num_ctx", 128000
                    ),  # Gemma 3 supports 128K context
                },
            )

            return response["response"].strip()
        except Exception as e:
            print(f"❌ Error processing image with Ollama: {e}")
            return ""


class VeterinaryTextAnalyzer:
    def __init__(
        self,
        llm_provider: str = "ollama",
        llm_model_name: str = "qwen2.5:7b",
        knowledge_base_path: Optional[str] = None,
        rebuild_vs=False,
        **llm_kwargs,
    ):
        print("🧠 Initializing VeterinaryTextAnalyzer...")
        self.llm_provider = llm_provider
        provider_instance = ModelProvider()
        self.llm = provider_instance.initialize_model(
            provider=llm_provider, model_name=llm_model_name, **llm_kwargs
        )

        print("🧠 Initializing VeterinaryTextAnalyzer...")
        self.kb_path = knowledge_base_path
        self.vector_store = VectorStoreManager(persist_path=self.kb_path)

        # 🔁 Always set up prompts/chains — even if not rebuilding
        self._setup_prompts()
        self._setup_chains()

        if rebuild_vs:
            self.vector_store.reset()
            self.rebuild_kb()

    def rebuild_kb(self, docs_path="./docs"):
        processor = VetDocumentProcessor()
        documents = processor.load_documents(docs_path)
        texts, embeddings, metadatas = processor.split_and_embed(documents)
        self.vector_store.add_documents(texts, embeddings, metadatas)
        print(f"✅ Rebuilt vector store with {len(texts)} chunks.")

        self._setup_prompts()
        self._setup_chains()

    def _setup_prompts(self):
        self.step1_prompt = ChatPromptTemplate.from_template(
            """
            You are a veterinary assistant reviewing blood test results for a dog.

            TASK:
            1. Parse each line for molecule name, value, and reference range.
            2. Compare the value to the normal range to determine if it is truly abnormal.
            3. Correct any mistaken evaluations.
            4. Return ONLY the truly abnormal results in this format:
               Molecule: VALUE (Normal: X-Y) – Status: High/Low

            Only include truly abnormal values.

            Raw Blood Test Results:
            {raw_text}
            """
        )

        self.step2_prompt = ChatPromptTemplate.from_template(
            """
            You are generating search queries for a veterinary knowledge base.

            Given the following abnormal blood results:

            {cleaned_results}

            Generate {top_results} distinct and relevant search queries that would help a veterinarian understand possible causes, implications, or treatments for these abnormalities in dogs.

            Format:
            1. query one
            2. query two
            ...
            """
        )

        self.step3_prompt = ChatPromptTemplate.from_template(
            """
            You are a veterinary assistant analyzing abnormal blood test results using additional medical knowledge.

            Abnormal Blood Results:
            {cleaned_results}

            Veterinary Knowledge:
            {context}

            TASK:
            1. Explain the clinical significance of each abnormal result.
            2. Suggest possible diagnoses or conditions.
            3. Recommend any next steps or follow-up.

            Provide a clear and professional analysis for a veterinarian in english.
            """
        )

        self.summary_prompt = ChatPromptTemplate.from_template(
            """
            Provide a 5-sentence summary based on the veterinary analysis below which is useful for the pet owner that does not have technical expertize about veterinary subject.

            Focus on:
            - Most concerning abnormalities
            - Potential diagnoses
            - Suggested next steps

            Veterinary Analysis:
            {analysis_results}

            Answer in english language.
            """
        )

    def _setup_chains(self):
        self.step1_chain = self.step1_prompt | self.llm | StrOutputParser()
        self.step2_chain = self.step2_prompt | self.llm | StrOutputParser()
        self.step3_chain = self.step3_prompt | self.llm | StrOutputParser()
        self.summary_chain = self.summary_prompt | self.llm | StrOutputParser()

    def _get_context(self, query: str) -> str:
        if self.vector_store:
            # vector_store.query now returns a string of concatenated docs, not a dict
            results = self.vector_store.query(query, top_k=5)
            return results  # results is already a string
        return ""

    def extract_and_correct_abnormal_results(self, raw_text: str) -> str:
        """Step 1: Extract and correct abnormal values from blood test results."""
        try:
            return self.step1_chain.invoke({"raw_text": raw_text})
        except Exception as e:
            print(f"❌ Step 1 failed: {e}")
            return "No valid abnormal results found."

    def generate_search_queries(self, cleaned_results: str) -> List[str]:
        """Step 2: Generate medical search queries from abnormal results."""
        try:
            raw_output = self.step2_chain.invoke({"cleaned_results": cleaned_results})
            return [
                line.strip("-•12345. ").strip()
                for line in raw_output.strip().splitlines()
                if line.strip()
            ]
        except Exception as e:
            print(f"❌ Step 2 failed: {e}")
            return []

    def analyze_cleaned_results_with_rag(
        self, cleaned_results: str, queries: List[str]
    ) -> str:
        try:
            context_blocks = [self._get_context(q) for q in queries]
            full_context = "\n\n".join(context_blocks)
            print("FULL CONTEXT")
            print(full_context)
            return self.step3_chain.invoke(
                {"cleaned_results": cleaned_results, "context": full_context}
            )
        except Exception as e:
            print(f"❌ Step 3 failed: {e}")
            return f"Analysis failed: {str(e)}"

    def generate_summary(self, analysis_results: str) -> str:
        """Optional: Generate concise summary of analysis."""
        try:
            return self.summary_chain.invoke({"analysis_results": analysis_results})
        except Exception as e:
            print(f"❌ Summary generation failed: {e}")
            return "Summary generation failed."

    def full_analysis_pipeline(self, raw_text: str) -> str:
        """
        Runs the full 3-step vet analysis pipeline and returns the final analysis and summary.
        """
        print("🔬 Step 1: Extracting abnormal results...")
        cleaned = self.extract_and_correct_abnormal_results(raw_text)

        print("🔍 Step 2: Generating search queries...")
        queries = self.generate_search_queries(cleaned)

        print("📚 Step 3: Performing analysis with RAG...")
        analysis = self.analyze_cleaned_results_with_rag(cleaned, queries)

        print("📝 Generating summary...")
        summary = self.generate_summary(analysis)

        return f"{analysis}\n\n🔎 Summary:\n{summary}"


class AiVetPipeline:
    """Complete pipeline for veterinary blood test analysis."""

    def __init__(
        self,
        vision_provider: str = "ollama",
        vision_model_name: str = "gemma3:4b",
        llm_provider: str = "ollama",
        llm_model_name: str = "qwen2.5:8b",
        knowledge_base_path: Optional[str] = None,
        vision_config: Optional[Dict] = None,
        **llm_kwargs,
    ):

        vision_config = vision_config or {}
        self.extractor = ImageToTextExtractor(
            vision_model_name=vision_model_name,
            vision_model_provider=vision_provider,
            **vision_config,
        )
        self.analyzer = VeterinaryTextAnalyzer(
            llm_provider, llm_model_name, knowledge_base_path, **llm_kwargs
        )
        print(
            f"🏥 AiVet Pipeline initialized with {vision_provider} vision and {llm_provider} LLM"
        )

    def process_blood_test(
        self,
        file_path: str,
        analysis_mode: str = "few_shot",
        generate_summary: bool = True,
    ) -> Dict[str, str]:
        """Complete processing pipeline for blood test analysis."""

        results = {}

        try:
            # Step 1: Extract text from image/PDF
            print("🔍 Step 1: Extracting text from image...")
            extracted_text = self.extractor.extract_text_from_file(file_path)
            results["extracted_text"] = extracted_text

            # Step 2: Analyze blood results
            print("🧪 Step 2: Analyzing blood test results...")
            analysis = self.analyzer.analyze_blood_results(
                extracted_text, analysis_mode
            )
            results["analysis"] = analysis

            # Step 3: Generate summary (optional)
            if generate_summary:
                print("📋 Step 3: Generating health summary...")
                summary = self.analyzer.generate_summary(analysis)
                results["summary"] = summary

            print("✅ Pipeline completed successfully")
            return results

        except Exception as e:
            print(f"❌ Pipeline failed: {e}")
            results["error"] = str(e)
            return results
        finally:
            # Clean up GPU memory
            self.extractor.clear_memory()

In [60]:
# pipeline_hf = AiVetPipeline(
#     vision_model_name="Qwen/Qwen2-VL-2B-Instruct",
#     vision_provider="huggingface",
#     llm_provider="qwen",
#     knowledge_base_path="./vet_knowledge_base/"
# )

# Example 2: Using Ollama Gemma 3 multimodal model
# pipeline_ollama = AiVetPipeline(
#     vision_model_name="qwen2.5vl:7b",  # or "gemma2:27b-vision" for larger model
#     vision_provider="ollama",
#     llm_provider="ollama",  # Using Ollama for text analysis too
#     llm_model_name="qwen2.5:7b",  # Using Ollama for text analysis too
#     knowledge_base_path="./vet_knowledge_base/",
#     vision_config={
#         "temperature": 0.1,
#         "top_k": 1,
#         "num_ctx": 32000,  # Take advantage of Gemma 3's 128K context
#     },
# )

# # Process a blood test image
# results = pipeline_ollama.process_blood_test(
#     file_path="../data/input/202402_hope_emo.pdf",
#     analysis_mode="few_shot",
#     generate_summary=True,
# )

# print("📊 Results:")
# for key, value in results.items():
#     print(f"\n{key.upper()}:\n{value}\n{'-'*50}")

In [61]:
context = """Here is the extracted information from the veterinary blood test report:

1. **RBC (Red Blood Cells)**
   - Detected Level: 7.18
   - Normal Range: 5.90 - 8.10 10^6/µl
   - Status: Normal

2. **HB (Hemoglobin)**
   - Detected Level: 15.6
   - Normal Range: 13.1 - 18.9 g/dl
   - Status: Normal

3. **HCT (Hematocrit)**
   - Detected Level: 44.6
   - Normal Range: 38.6 - 54.5 %
   - Status: Normal

4. **MCV (Mean Corpuscular Volume)**
   - Detected Level: 62.1
   - Normal Range: 61 - 72.6 fL
   - Status: Normal

5. **MCH (Mean Corpuscular Hemoglobin)**
   - Detected Level: 22.0
   - Normal Range: 20 - 26 pg
   - Status: Normal

6. **MCHC (Mean Corpuscular Hemoglobin Concentration)**
   - Detected Level: 35.00
   - Normal Range: 30 - 37 g/dl
   - Status: Normal

7. **CH (Cholesterol)**
   - Detected Level: 21.1
   - Normal Range: 21.0 - 26.5 mg/dl
   - Status: Normal

8. **CHDW (Cholesterol Distribution Width)**
   - Detected Level: 2.79
   - Normal Range: 1.7 - 3.80 mg/dl
   - Status: Normal

9. **HDW (Hemoglobin Distribution Width)**
   - Detected Level: 1.7
   - Normal Range: 1.60 - 2.27 g/dl
   - Status: Normal

10. **RDW (Red Cell Distribution Width)**
    - Detected Level: 13.8
    - Normal Range: 10.9 - 14.5 %
    - Status: Normal

11. **WBC (White Blood Cells)**
    - Detected Level: 13.88
    - Normal Range: 5.70 - 13.80 10^3/µl
    - Status: Normal

12. **Neutrophils (Neut)**
    - Detected Level: 11381.6
    - Normal Range: 3800 - 8800/µl
    - Status: Normal

13. **Lymphocytes (Linf)**
    - Detected Level: 1665.6
    - Normal Range: 1300 - 4100/µl
    - Status: Normal

14. **Monocytes (Mono)**
    - Detected Level: 694.0
    - Normal Range: 200 - 740/µl
    - Status: Normal

15. **Eosinophils (Eos)**
    - Detected Level: 138.8
    - Normal Range: 150 - 1100/µl
    - Status: Normal

16. **Basophils (Baso)**
    - Detected Level: 0
    - Normal Range: 0 - 100/µl
    - Status: Normal

17. **Platelets (PLT)**
    - Detected Level: 101
    - Normal Range: 150 - 460 1000/µl
    - Status: Low

18. **MPV (Mean Platelet Volume)**
    - Detected Level: 9.7
    - Normal Range: 8.6 - 18 fL
    - Status: Normal

19. **PCT (Platelet Crit)**
    - Detected Level: 0.10
    - Normal Range: 0.24 - 0.70
    - Status: Low

20. **MPC (Mean Platelet Count)**
    - Detected Level: 26.20
    - Normal Range: 53 - 70
    - Status: Normal

21. **MPD (Mean Platelet Distribution Width)**
    - Detected Level: 57.50
    - Normal Range: 53 - 70 %
    - Status: Normal

22. **LPLT (Low Platelet Level)**
    - Detected Level: 4
    - Normal Range: 1 - 50 1000/µl
    - Status: Low

23. **Plasma Opalescence**
    - Detected Level: Plasma opalescent
    - Status: Normal
--------------------------------------------------"""

In [62]:
# # Define your paths
# docs_path = "../vetdb"  # e.g. folder with PDFs or .txts
# persist_path = "./chroma_db"  # folder where Chroma stores DB

# # Optional: delete existing and rebuild
# overwrite = True

# # 🔄 Step 1: Reset Chroma if rebuilding
# vector_store = VectorStoreManager(persist_path=persist_path)
# # if overwrite:
# #     vector_store.reset()

# # 📄 Step 2: Load, split, embed
# processor = VetDocumentProcessor()
# docs = processor.load_documents(docs_path)
# texts, embeddings, metadatas = processor.split_and_embed(docs)

# # 💾 Step 3: Store in vector DB
# vector_store.add_documents(texts, embeddings, metadatas)
# print("✅ Vector store built and saved.")

In [63]:
# import py7zr

# archive_path = "../vetdb/Veterinary Hematology, A Diagnostic Guide and Color Atlas (VetBooks.ir).7z"
# extract_path = "../vetdb"
# password = "vetbooks.ir"

# with py7zr.SevenZipFile(archive_path, mode='r', password=password) as archive:
#     archive.extractall(path=extract_path)

# print("Extraction complete!")

In [64]:
# if os.path.exists("./chroma_db"):
#     shutil.rmtree("./chroma_db")

In [65]:
# builder = VetVectorStoreBuilder()
# builder.build_vectorstore(
#     docs_path="../vetdb",  # folder with PDFs/text
#     persist_path="./chroma_db",  # where to store Chroma vectors
#     overwrite=True,  # set False to append
# )

In [66]:
# --- Simulated blood test result ---
blood_results = context
# --- Setup: With RAG ---
print("🔍 Testing with RAG enabled...")
analyzer_with_rag = VeterinaryTextAnalyzer(
    llm_provider="ollama",
    llm_model_name="qwen2.5:7b",
    knowledge_base_path="./chroma_db",  # Path to your veterinary knowledge base (folder or file)
)
print(
    f"Total documents in vector store: {len(analyzer_with_rag.vector_store.collection.get()['ids'])}"
)

print("🔬 Step 1: Extracting abnormal results...")
cleaned = analyzer_with_rag.extract_and_correct_abnormal_results(blood_results)

print("🔍 Step 2: Generating search queries...")
queries = analyzer_with_rag.generate_search_queries(cleaned)

print("📚 Step 3: Performing analysis with RAG...")
analysis = analyzer_with_rag.analyze_cleaned_results_with_rag(cleaned, queries)

🔍 Testing with RAG enabled...
🧠 Initializing VeterinaryTextAnalyzer...
ollama
qwen2.5:7b
✅ Ollama multimodal model qwen2.5:7b ready
🧠 Initializing VeterinaryTextAnalyzer...
Total documents in vector store: 2265
🔬 Step 1: Extracting abnormal results...
🔍 Step 2: Generating search queries...
📚 Step 3: Performing analysis with RAG...
FULL CONTEXT
erythrocytosis (polycythemia vera) in cats.
434,481 Essential 
thrombocythemia has been diagnosed in animals with persis-
tent markedly elevated platelet counts, typically in excess of  
1 × 106/µL (see Fig. 7-35), for which a cause for a reactive 
thrombocytosis could not be identified. Platelet morphology 
is typically normal; however, increased MPVs have been 
reported in two dogs believed to have thrombocythemia.
147 
Essential thrombocythemia may be viewed as the platelet 
counterpart of primary erythrocytosis (see Chapter 9 for more 
information).
COAGULATION DISORDERS
Acquired Coagulation Disorders
Hypercoagulable State
The increased tende

In [67]:
print(cleaned)

Molecule: Platelets (PLT) – Status: Low  
Molecule: PCT (Platelet Crit) – Status: Low


In [68]:
print(queries)

['Query one: "Low platelets (PLT) and PCT in dogs - causes"', 'Query two: "Implications of low platelet count (PLT) and PCT for canine health"', 'Query three: "Treatment options for dogs with low platelet levels (PLT) and PCT"', 'Query four: "Common diseases leading to low PLT and PCT in canines"', 'Query five: "Diagnostic tests for low platelets and PCT in dogs"']


In [69]:
print(analysis)

### Analysis of Abnormal Blood Test Results

#### 1. Clinical Significance of Each Abnormal Result

**Platelets (PLT) – Status: Low**
- **Clinical Significance:** A low platelet count, or thrombocytopenia, indicates that the pet has a reduced number of platelets in their blood. Platelets are crucial for blood clotting and preventing excessive bleeding.
  - **Potential Causes:**
    - **Immune-Mediated Thrombocytopenia (IMT):** The immune system mistakenly attacks and destroys platelets, leading to a low count.
    - **Hypersplenism:** An enlarged spleen can remove more platelets from the circulation than normal.
    - **Bone Marrow Disorders:** Conditions that affect bone marrow function can lead to reduced platelet production.
    - **Surgical Procedures:** Platelets may be utilized or destroyed during surgery, causing temporary thrombocytopenia.

**PCT (Platelet Crit) – Status: Low**
- **Clinical Significance:** PCT measures the percentage of blood volume occupied by platelets. A low

In [70]:
summary = analyzer_with_rag.generate_summary(analysis)

In [71]:
summary

"1. The most concerning abnormalities in the blood test include a low platelet count (thrombocytopenia) and a low plateletcrit, which may indicate immune-mediated thrombocytopenia, hypersplenism, or bone marrow disorders.\n2. Potential diagnoses include Immune-Mediated Thrombocytopenia (IMT), characterized by bleeding symptoms like nosebleeds and gum bleeding, Hypersplenism, often associated with other blood abnormalities, and Bone Marrow Disorders, which can cause additional hematological issues.\n3. Suggested next steps involve confirming the quality of current blood samples, conducting further tests such as a complete blood count (CBC), coagulation function tests, and spleen ultrasound, along with detailed patient history and physical examination.\n4. Additional specific tests recommended include anti-platelet antibody detection to rule out immune-mediated causes, and bone marrow aspiration or biopsy if necessary to diagnose bone marrow disorders.\n5. Regular follow-up is advised to